In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年4月17日～

## モデル$ T = aXY + bXZ + cYZ + d $に関する実装
TODO
* モデル式 $ T = aXY + bXZ + cYZ + d $ においてMAPE表を作成する（Model_InfiniteProductOfProblemSizeMultipliedByProcesses_ForMultipleRegression）
    * Modelsクラスに追加実装
    * 表への追加

（T,X,Y,Zはそれぞれ、関数コール回数,プロセス数,問題サイズ1,問題サイズ2）
（a,b,cは係数, dは切片）